# Event Report

> Use this application to get team statistics for CurlingZone covered events.

## How
> Enter the CurlingZone event ID corresponding to the event you're interested in and Run Report! button and wait 30 seconds - 1 minute for the results to be generated. The CurlingZone event ID can be found in the CurlingZone URL, e.g. https\://curlingzone.com/event.php?**eventid=7847**&view=Scores&showdrawid=10#1.

## What

The following statistics are generated by the app:

* \# of games played.
* Hammer to start game pct (# games starting with hammer / # games played).
* Avg. points scored (points scored / # games played).
* More to come...

In [ ]:
import czapi.api as api
import pandas as pd
from time import sleep
import ipywidgets

In [ ]:
columns = ['team_name','href','hammer_start','score','final_score','hammer_progression','relative_score','guid']
def make_frame(event:api.Event)->pd.DataFrame:
    return pd.concat([pd.DataFrame(data=boxscore,columns=columns) for boxscore in event.get_flat_boxscores()],axis=0,sort=True).reset_index(drop=True)


In [ ]:
def explode_frame(df:pd.DataFrame)->pd.DataFrame:
    df['end'] = df.hammer_progression.apply(lambda x: range(1,len(x)+1))
    return df.explode(column=['hammer_progression','relative_score','end']).reset_index(drop=True)

In [ ]:
def transform_frame(df:pd.DataFrame)->pd.DataFrame:
    return df.assign(

     final_score = df.final_score.astype(int)
    ,hammer_progression = df.hammer_progression.astype(bool)
    ,hammer_start = df.hammer_start.astype(bool)
    ,relative_score = df.relative_score.astype(int)
    ,ends = df.end.astype(int)
    ,team_id = df.href.str.extract(pat='^.*teamid=(\d*)[#&].*').astype(int)

)


In [ ]:
def get_games_played(df:pd.DataFrame)->pd.DataFrame:
    return df.groupby(['team_name','team_id'])['guid'].nunique().sort_values(ascending=False).to_frame().rename({'guid':'games_played'},axis=1)


In [ ]:
def get_hammer_to_start_game(df:pd.DataFrame)->pd.DataFrame:

    return df[['team_name','team_id','guid','hammer_start']].drop_duplicates().groupby(['team_name','team_id'])['hammer_start'].mean().sort_values(ascending=False).to_frame().rename({'hammer_start':'pct_start_with_hammer'},axis=1)



In [ ]:
def get_points_scored(df:pd.DataFrame)->pd.DataFrame:

    return df[['team_name','team_id','guid','final_score']].drop_duplicates().groupby(['team_name','team_id'])['final_score'].mean().sort_values(ascending=False).to_frame().rename({'final_score':'pts_for'},axis=1)


In [ ]:
print('Input CurlingZone event ID here:')
event_id = ipywidgets.widgets.BoundedIntText(min=1,max=10000)
event_id

In [ ]:
run = ipywidgets.widgets.Button(description="Run report!")
out = ipywidgets.widgets.Output()

def run_report(change)->api.Event:
    
    out.clear_output()
    
    out.append_stdout('Getting data from CurlingZone...Please wait.')
    try:
        event = api.Event(cz_event_id = event_id.value,verbose=True,delay=3)
    except Exception:
        out.append_stdout('\nBad Event ID, try again please.')
        return
    df = make_frame(event=event)
    df = explode_frame(df=df)
    df = transform_frame(df=df)
    

    
    # games played
    games_played = get_games_played(df=df)
    out.append_display_data(games_played)
    
    # hammer to start game
    hammer_to_start_game = get_hammer_to_start_game(df=df)
    out.append_display_data(hammer_to_start_game)
    
    # points score
    
    points_scored = get_points_scored(df=df)
    out.append_display_data(points_scored)
    
#     print(df.head())
#     print(df.dtypes)

run.on_click(run_report)

In [ ]:
ipywidgets.widgets.VBox([run,out])